# Setup configurations

In [1]:
# Setup parameter values
gt_input_dataset_name = "bfp_detection_ground_truth"
default_data_path = "data/"
datastore_path = f"data/"


## Connect to AzureML workspace

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("Workspace", ws.name, "loaded")

## Prepare environment

In [ ]:
from azureml.core import Environment
env_name = 'poi-bfp-detection_efficientnetb7_training'
data_preparation_env = Environment.from_conda_specification(env_name, f'./../requirements/{env_name}.yaml')

# Specify a GPU base image
# training_segmentation_env.docker.enabled = True
data_preparation_env.docker.shared_volumes = False

data_preparation_env.docker.base_image = None
data_preparation_env.docker.base_dockerfile = "../requirements/Dockerfile"

# register 
data_preparation_env.register(workspace=ws)

data_preparation_env = Environment.get(ws, env_name)
print('Environment', data_preparation_env.name, 'registered. Version', data_preparation_env.version)

No Python version provided, defaulting to "3.8.13"
'shared_volumes' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object instead.


Environment poi-bfp-detection_efficientnetb7_training registered. Version 3


## Define run configuration

In [ ]:
from azureml.core.runconfig import RunConfiguration,DockerConfiguration
docker_configuration = DockerConfiguration(use_docker=True)

monitoring_run_config = RunConfiguration()
monitoring_run_config.docker = docker_configuration
monitoring_run_config.environment = data_preparation_env

## Prepare compute target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'Torch-computing'

compute_target = AmlCompute(ws, compute_target_name)
print(f'Existing compute target {compute_target_name} found.')

Existing compute target Torch-computing found.


## Register experiment

In [ ]:
from azureml.core import Experiment

experiment_name = 'expr-bfp-detection_efficientnetb7_training'
experiment = Experiment(workspace=ws, name=experiment_name)

## Create and register Dataset

In [ ]:
from azureml.core import Datastore, Dataset

datastores = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name='poimachinelearning',
    container_name='bfp-detection',
    account_name='poimachinelearning',
    account_key='p4EEQkYTnq4jkfVtyAC2iKhVwGRSP96AumqVC8YXAzxU6h3r3Ns/5L5FuKqm3R4WtxgfPyOhfC+3lGnSgimQSA==',
)

In [ ]:
# datastore = ws.get_default_datastore()
datastore = ws.datastores['poimachinelearning']
datastore_paths = [(datastore,datastore_path)]

input_dataset = Dataset.File.from_files(path=datastore_paths)
input_dataset.register(workspace = ws,name = gt_input_dataset_name,
                       description = 'Ground truth data for building footprint detection',
                       create_new_version = True)

{
  "source": [
    "('poimachinelearning', 'data/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "49a4b873-96e7-49d5-b040-b696430a1268",
    "name": "bfp_detection_ground_truth",
    "version": 2,
    "description": "Ground truth data for building footprint detection",
    "workspace": "Workspace.create(name='poi-ml-test', subscription_id='4053f59e-e58c-425d-90d1-e738fb166047', resource_group='sb-ml-dev')"
  }
}

## Building the pipeline

### Define pipeline parameters

In [ ]:
from azureml.pipeline.core.graph import PipelineParameter

input_dataset = PipelineParameter(name="path",default_value=default_data_path)
input_size = PipelineParameter(name="imgsize",default_value=512)
classes = PipelineParameter(name='classes',default_value=1)
# model params 
epochs = PipelineParameter(name="epochs",default_value=50)
loss = PipelineParameter(name="loss",default_value="jaccard_distance")
metrics = PipelineParameter(name='metrics',default_value='all')
backbone= PipelineParameter(name='backbone',default_value='efficientnetb7')
batch_size = PipelineParameter(name='batch_size',default_value=1)
out_path = PipelineParameter(name='out_path',default_value='runs')


## Define Pipleine Steps 

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

bfp_detection_script_args = [
    "--path", input_dataset,
    "--imgsize", input_size,
    "--classes", classes,
    "--epochs",epochs,
    "--loss",loss,
    "--metrics",metrics,
    "--backbone",backbone,
    "--batch_size",batch_size,
    "--out_path",out_path
]

bfp_detection_training_step=PythonScriptStep(
    name='POI Building footprint detection',
    script_name='main.py',
    arguments=bfp_detection_script_args,
    compute_target=compute_target,
    runconfig=monitoring_run_config,
    source_directory='../scripts/'
)

## Create pipeline

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[bfp_detection_training_step])
print("Pipeline created")

Pipeline created


### Submit pipeline experiment

In [ ]:
from azureml.widgets import RunDetails

pipeline_run = experiment.submit(pipeline)
RunDetails(pipeline_run).show()

Created step POI Building footprint detection [f8e973d7][81799387-4f00-4f86-85a6-c92ebc1f2b85], (This step will run and generate new outputs)
Submitted PipelineRun cd03b22e-9ef0-4e4c-a975-f829440656d1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cd03b22e-9ef0-4e4c-a975-f829440656d1?wsid=/subscriptions/4053f59e-e58c-425d-90d1-e738fb166047/resourcegroups/sb-ml-dev/workspaces/poi-ml-test&tid=374f8026-7b54-4a3a-b87d-328fa26ec10d


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

### Validate pipeline

In [ ]:
pipeline.validate()
print("Pipeline validation completed")